In [1]:
# %%time
# !pip install --target=/kaggle/working --extra-index-url=https://pypi.nvidia.com "cudf-cu12==25.2.*" "cuml-cu12==25.2.*"
# !rm -rf /kaggle/working/numpy*

In [2]:
# !pip install scikit-learn==1.5.2 koolbox



In [3]:
# import cudf
# cudf.__version__

In [4]:

# import cuml
# cuml

In [5]:
# %load_ext cudf.pandas

import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import category_encoders as ce

pd.set_option('display.max_columns', 500)

VER=1

In [6]:
# from cuml.preprocessing import TargetEncoder
from sklearn.model_selection import KFold
import xgboost as xgb

print(f"XGBoost version {xgb.__version__}")

XGBoost version 2.0.3


In [7]:
# from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb
from xgboost import XGBClassifier
from scipy.special import logit
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import joblib
import shutil
import optuna
import glob
import json

warnings.filterwarnings('ignore')

In [8]:
train=pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv',index_col='id')
test=pd.read_csv('/kaggle/input/playground-series-s5e8/test.csv',index_col='id') 
# augment_data = pd.read_csv('/kaggle/input/data-augmentation/df_synth.csv')
# train=train.sample(frac=0.1).reset_index(drop=True)
# test=test.sample(frac=0.1).reset_index(drop=True)
train.shape
test['y'] = -1

In [9]:
orig = pd.read_csv("/kaggle/input/bank-marketing-dataset-full/bank-full.csv",delimiter=";") 
orig['y'] = orig.y.map({'yes':1,'no':0})
orig['id'] = (np.arange(len(orig))+1e6).astype('int')
orig = orig.set_index('id')
print("Original data shape", orig.shape )
orig.head()

Original data shape (45211, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
id,,,,,,,,,,,,,,,,,
1000000,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,0
1000001,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,0
1000002,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,0
1000003,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,0
1000004,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,0


In [10]:
combine = pd.concat([train,test,orig],axis=0)


In [11]:
# train2.y=train2.y.map({"no":0,'yes':1})

In [12]:
# augment_data.columns=train.columns

In [13]:
# from sklearn.preprocessing import LabelEncoder
# le=LabelEncoder()
# df=pd.DataFrame()
# dict_lab=dict()
# for col in train2.select_dtypes(include='object').columns:
#     df[col] = le.fit_transform(train2[col])
#     label_dict = {label: idx for idx, label in enumerate(le.classes_)}
#     dict_lab[col]=label_dict

# for key in dict_lab.keys():
#     augment_data[key]=augment_data[key].map({k:v for v,k in dict_lab[key].items()})


# train3=augment_data[augment_data.y==1]
# train = pd.concat([train,train2,train3],axis=0,ignore_index=True).sample(frac=1.0).reset_index(drop=True)
# print("Combined Train shape", train.shape)

In [14]:
def data_process(df): # Source: https://www.kaggle.com/code/haohuanchen/ps-s5e8-lightgb-model-a-simple-starter
    df = df.copy()
    
    def many_no(x):
        if x['default']=='no' and x['housing']=='no' and x['loan']=='no':
            return 21
        if x['default']=='no' and x['housing']=='no'\
        or x['default']=='no' and x['loan']=='no'\
        or x['housing']=='no' and x['loan']=='no':
            return 7
        if x['default']=='no' or x['housing']=='no' or x['loan']=='no':
            return 3
        return 0
    df['many_no']  = df.apply(lambda x: many_no(x), axis=1).astype("str")
    return df

def data_process2(df):
    df = df.copy()
    
    def credit(x):
        credit_level = (x['default'] == 'no') + (x['housing'] == 'no') + (x['loan'] == 'no')
        return {3: 27, 2: 9, 1: 3}.get(credit_level, 0)
    df['credit'] = df.apply(lambda x: credit(x), axis=1)
    
    def risk(x):
        risk_level = (x['default'] == 'yes') + (x['housing'] == 'yes') + (x['loan'] == 'yes')
        return {3: 27, 2: 9, 1: 3}.get(risk_level, 0)
    df['risk'] = df.apply(lambda x: risk(x), axis=1)
    
    df['balance_log_i'] = df['balance'].where(df['balance'] > -1, 0).apply(np.log1p)
    df['duration_log_i'] = df['duration'].where(df['duration'] > -1, 0).apply(np.log1p)
    
    df['balance_log'] = df['balance'].where(df['balance'] > -1, 0).apply(np.cbrt)
    df['duration_log'] = df['duration'].where(df['duration'] > -1, 0).apply(np.cbrt)
    # df['campaign_log'] = df['campaign'].where(df['campaign'] > -1, 0).apply(np.log1p)
    # df['previous_log'] = df['previous'].where(df['previous'] > -1, 0).apply(np.log1p)
    
    # month_map = {'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6, 'jul':7,'aug':8,'sep':9,'oct':10,'nov':11,'dec':12}
    # df['month'] = df['month'].map(month_map)
  
     
    return df

 
 
def NEW_FE(df):
    
    df['balance_log'] = np.log1p(df['balance'].clip(lower=0))
    df['job_edu'] = df['job'].astype(str) + "_" + df['education'].astype(str)
    df['contacted_before'] = (df['pdays'] != -1).astype(int)
    df['age_squared'] = df['age'] ** 2

    df['duration_sin'] = np.sin(2*np.pi * df['duration'] / 400)
    df['duration_cos'] = np.cos(2*np.pi * df['duration'] / 400)

    # month_map = {
    # 'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4,
    # 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8,
    # 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
    # }
    # df['month_num'] = df['month'].map(month_map).astype('int')

    # df['month_sin'] = np.sin(2 * np.pi * df['month_num'] / 12)
    # df['month_cos'] = np.cos(2 * np.pi * df['month_num'] / 12)

    # df.drop('month_num',axis=1,inplace=True)
    
    return df

 
def assign_group(age):
        thresholds = [0,27.625,37.25, 46.875, 56.5,   66.125, 75.75, 185.375, 95.0]
        for i, j in enumerate(thresholds):
            if age <= j:
                return i
        return np.nan

 
def assign_group2(age):
        thresholds =[-8019. ,  2754.6, 13528.2, 24301.8, 35075.4, 45849. , 56622.6,
       67396.2, 78169.8, 88943.4, 99717. ]
        for i, j in enumerate(thresholds):
            if age <= j:
                return i
        return np.nan
 


In [15]:
CATS = []
NUMS = []
for c in combine.drop('y',axis=1).columns:
    t = "CAT"
    if combine[c].dtype=='object':
        CATS.append(c)
    else:
        NUMS.append(c)
        t = "NUM"
    n = combine[c].nunique()
    na = combine[c].isna().sum()
    print(f"[{t}] {c} has {n} unique and {na} NA")
print("CATS:", CATS )
print("NUMS:", NUMS )

[NUM] age has 78 unique and 0 NA
[CAT] job has 12 unique and 0 NA
[CAT] marital has 3 unique and 0 NA
[CAT] education has 4 unique and 0 NA
[CAT] default has 2 unique and 0 NA
[NUM] balance has 8590 unique and 0 NA
[CAT] housing has 2 unique and 0 NA
[CAT] loan has 2 unique and 0 NA
[CAT] contact has 3 unique and 0 NA
[NUM] day has 31 unique and 0 NA
[CAT] month has 12 unique and 0 NA
[NUM] duration has 1824 unique and 0 NA
[NUM] campaign has 52 unique and 0 NA
[NUM] pdays has 628 unique and 0 NA
[NUM] previous has 54 unique and 0 NA
[CAT] poutcome has 4 unique and 0 NA
CATS: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
NUMS: ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']


In [16]:
CATS1 = []
SIZES = {}
for c in NUMS + CATS:
    n = c
    if c in NUMS: 
        n = f"{c}2"
        CATS1.append(n)
    combine[n],_ = combine[c].factorize()
    SIZES[n] = combine[n].max()+1

    combine[c] = combine[c].astype('int32')
    combine[n] = combine[n].astype('int32')
# numireic but encoding why!!!!!
print("New CATS:", CATS1 )
print("Cardinality of all CATS:", SIZES )

New CATS: ['age2', 'balance2', 'day2', 'duration2', 'campaign2', 'pdays2', 'previous2']
Cardinality of all CATS: {'age2': 78, 'balance2': 8590, 'day2': 31, 'duration2': 1824, 'campaign2': 52, 'pdays2': 628, 'previous2': 54, 'job': 12, 'marital': 3, 'education': 4, 'default': 2, 'housing': 2, 'loan': 2, 'contact': 3, 'month': 12, 'poutcome': 4}


In [17]:
from itertools import combinations

pairs = combinations(CATS + CATS1, 2)
new_cols = {}
CATS2 = []

for c1, c2 in pairs:
    s = SIZES[c1] * SIZES[c2]
    name = "_".join(sorted((c1, c2)))
    new_cols[name] = combine[c1] * SIZES[c2] + combine[c2]
    CATS2.append(name)
if new_cols:
    new_df = pd.DataFrame(new_cols)         
    combine = pd.concat([combine, new_df], axis=1) 

print(f"Created {len(CATS2)} new CAT columns")

Created 120 new CAT columns


In [18]:
combine['contact_housing_month']= combine['contact']* SIZES["housing"]+combine['housing']* SIZES["month"]+combine['month']
 
DD=[
 
    "contact_housing_month",
 
]

In [19]:

Top_20_Features= ['duration2_poutcome_previous2', 
                          'contact_duration2_pdays2', 
                          'duration2_pdays2_poutcome',
                          'contact_month_poutcome',
                          'contact_duration2_previous2', 
                          'housing_month_previous2',
                          'contact_job_loan',
                          'duration2_housing_pdays2',
                          'day2_month_poutcome', 
                          'campaign2_contact_duration2', 
                          'contact_day2_month', 
                          'contact_month_previous2',
                          'default_housing_loan',
                          'contact_housing_loan', 
                          'balance2_poutcome_previous2',
                          'balance2_pdays2_poutcome']
for top in Top_20_Features:
    t1=top.split("_")[0]
    t2=top.split("_")[1]
    t3=top.split("_")[2]
    combine[top]= combine[t1]** SIZES[t2]+combine[t2]* SIZES[t2]+combine[t3]

In [20]:
CE = []
CC = CATS+CATS1+CATS2+DD

print(f"Processing {len(CC)} columns... ",end="")
for i,c in enumerate(CC):
    if i%10==0: print(f"{i}, ",end="")
    tmp = combine.groupby(c).y.count()
    tmp = tmp.astype('int32')
    tmp.name = f"CEE_{c}"
    CE.append( f"CEE_{c}" )
    combine = combine.merge(tmp, on=c, how='left')
print()

Processing 137 columns... 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 


In [21]:
train = combine.iloc[:len(train)]
test = combine.iloc[len(train): len(train)+len(test)]
orig = combine.iloc[-len(orig):]
del combine
print("Train shape", train.shape,"Test shape", test.shape ,"Orig shape", orig.shape  )

Train shape (750000, 298) Test shape (250000, 298) Orig shape (45211, 298)


In [22]:
test = test.drop('y',axis=1)


In [23]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold,KFold
import numpy as np


In [24]:
# FEATURES = CATS+CATS1+CATS2+CE + ["balance"]  + Num

# print(f"We now have {len(FEATURES)} columns:")
# unique_list = list(set(FEATURES))
# print(len(unique_list))
# FEATURES=unique_list

In [25]:
TE_ORIG = []
CC = CATS+CATS1+CATS2 

print(f"Processing {len(CC)} columns... ",end="")
for i,c in enumerate(CC):
    if i%10==0: print(f"{i}, ",end="")
    tmp = orig.groupby(c).y.mean()
    tmp = tmp.astype('float32')
    tmp.name = f"TE_ORIG_{c}"
    TE_ORIG.append( f"TE_ORIG_{c}" )
    train = train.merge(tmp, on=c, how='left')
    test = test.merge(tmp, on=c, how='left')
    orig = orig.merge(tmp, on=c, how='left')
print()

Processing 136 columns... 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 


In [26]:
import torch

# Clear cache
torch.cuda.empty_cache()

# Optionally free all tensors not referenced anymore
import gc
gc.collect()
torch.cuda.empty_cache()


In [27]:
# STATISTICS TO AGGEGATE FOR OUR FEATURE GROUPS
STATS = ["mean","std","count","nunique","median","min","max","skew", "prod"]
STATS2 = ["mean","std"]

In [28]:
scores = {}
oof_pred_probs = {}
test_pred_probs = {}

In [29]:
cat_features_card = []
for f in CATS:
        cat_features_card.append(train[f].nunique())

In [30]:
AUGM=[
   'many_no',
    "credit",
    "risk",
    "balance_log_i",
    "duration_log_i",
    "balance_log",
    "duration_log",
    "job_edu",
    "contacted_before",
    "age_squared",
    "duration_sin",
    "duration_cos",
     "group_age",
    "group_balance"
   ]


train = data_process(train)
train["group_age"] = train['age'].apply(assign_group)
train["group_balance"] = train['balance'].apply(assign_group2)
train = data_process2(train) 
train =NEW_FE(train)

test = data_process(test)
test["group_age"] = test['age'].apply(assign_group)
test["group_balance"] = test['balance'].apply(assign_group2)
test = data_process2(test) 
test =NEW_FE(test)

orig = data_process(orig)
orig["group_age"] = orig['age'].apply(assign_group)
orig["group_balance"] = orig['balance'].apply(assign_group2)
orig = data_process2(orig) 
orig =NEW_FE(orig)

from sklearn.preprocessing import LabelEncoder

for col in ["many_no", "job_edu"]:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col].astype(str))
    test[col] = le.transform(test[col].astype(str))

 





train['group_balance']=train['group_balance'].fillna(train['group_balance'].median())       
test['group_balance']=test['group_balance'].fillna(test['group_balance'].median())       
orig['group_balance']=orig['group_balance'].fillna(orig['group_balance'].median())

      

In [31]:
FEATURES = NUMS+CATS+CATS1+CATS2+CE+TE_ORIG+Top_20_Features+DD+AUGM
print(f"We have {len(FEATURES)} features.")


We have 447 features.


In [32]:
train.to_csv('train_xglm.csv',index=False)
test.to_csv('test_xglm.csv',index=False)
orig.to_csv('orig_xglm.csv',index=False)

In [33]:
# train = pd.concat([train],axis=0).reset_index(drop=True)
 

In [34]:
train.y=train.y.astype(int)
# from cuml.preprocessing import TargetEncoder
# del orig

In [35]:
FOLDS =11
SEED = 42

params = {
    "objective": "binary:logistic",  
    "eval_metric": "auc",           
    "learning_rate": 0.1,
    "max_depth": 0,
    "subsample": 0.8,
    "colsample_bytree": 0.7,
    "seed": SEED,
    "device": "cuda",
    "grow_policy": "lossguide", 
    "max_leaves": 32,          
    "alpha": 2.0,
}

In [36]:
class IterLoadForDMatrix(xgb.core.DataIter):
    def __init__(self, df=None, features=None, target=None, batch_size=256*1024):
        self.features = features
        self.target = target
        self.df = df
        self.it = 0 
        self.batch_size = batch_size
        self.batches = int( np.ceil( len(df) / self.batch_size ) )
        super().__init__()

    def reset(self):
        '''Reset the iterator'''
        self.it = 0

    def next(self, input_data):
        '''Yield next batch of data.'''
        if self.it == self.batches:
            return 0 # Return 0 when there's no more batch.
        
        a = self.it * self.batch_size
        b = min( (self.it + 1) * self.batch_size, len(self.df) )
        #dt = cudf.DataFrame(self.df.iloc[a:b])
        dt = self.df.iloc[a:b]
        input_data(data=dt[self.features], label=dt[self.target]) 
        self.it += 1
        return 1

In [37]:
# oof_preds = np.zeros(len(train))
# test_preds = np.zeros(len(test))


# kf = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
# for fold, (train_idx, val_idx) in enumerate(kf.split(train)):
#     print("#"*25)
#     print(f"### Fold {fold+1} ###")
#     print("#"*25)

#     Xy_train = train.iloc[train_idx][ FEATURES+['y'] ].copy()
#     Xy_more = orig[ FEATURES+['y'] ]
#     for k in range(1):
#         Xy_train = pd.concat([Xy_train,Xy_more],axis=0,ignore_index=True)
    
#     X_valid = train.iloc[val_idx][FEATURES].copy()
#     y_valid = train.iloc[val_idx]['y']
#     X_test = test[FEATURES].copy()

#     CC = CATS1+CATS2+DD
#     print(f"Target encoding {len(CC)} features... ",end="")
#     for i,c in enumerate(CC):
#         if i%10==0: print(f"{i}, ",end="")
#         TE0 = TargetEncoder(n_folds=10, smooth=0, split_method='random', stat='mean')
#         Xy_train[c] = TE0.fit_transform(Xy_train[c],Xy_train['y']).astype('float32')
#         X_valid[c] = TE0.transform(X_valid[c]).astype('float32')
#         X_test[c] = TE0.transform(X_test[c]).astype('float32')
        
#         torch.cuda.empty_cache()

#         # Optionally free all tensors not referenced anymore
#         gc.collect()
#         torch.cuda.empty_cache()
#     print()

#     Xy_train[CATS] = Xy_train[CATS].astype('category')
#     X_valid[CATS] = X_valid[CATS].astype('category')
#     X_test[CATS] = X_test[CATS].astype('category')

#     Xy_train = IterLoadForDMatrix(Xy_train, FEATURES, 'y')
#     dtrain = xgb.QuantileDMatrix(Xy_train, enable_categorical=True, max_bin=256)
#     dval   = xgb.DMatrix(X_valid, label=y_valid, enable_categorical=True)
#     dtest  = xgb.DMatrix(X_test, enable_categorical=True)

#     model = xgb.train(
#         params=params,
#         dtrain=dtrain,
#         num_boost_round=10_000,
#         evals=[(dtrain, "train"), (dval, "valid")],
#         early_stopping_rounds=200,
#         verbose_eval=200
#     )

#     oof_preds[val_idx] = model.predict(dval, iteration_range=(0, model.best_iteration + 1))
#     test_preds += model.predict(dtest, iteration_range=(0, model.best_iteration + 1)) / FOLDS

In [38]:
# from sklearn.metrics import roc_auc_score

# m = roc_auc_score(train.y, oof_preds)
# print(f"XGB with Original Data as rows CV = {m}")
#  # XGB with Original Data as rows CV = 0.969810900952446


In [39]:
%%time
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)
def train_model ( mode='xgb'):
    oof = np.zeros((len(train)))
    pred = np.zeros((len(test)))
    save_data=dict()
   
        
    # OUTER K FOLD
    for i, (train_index, test_index) in enumerate(kf.split(train)):
        print(f"### OUTER Fold {i+1} ###")
      
        
        
        X_train = train.loc[train_index,FEATURES+['y']].reset_index(drop=True).copy()
        y_train = train.loc[train_index,'y']
        
        
        X_valid = train.loc[test_index,FEATURES].reset_index(drop=True).copy()
        y_valid = train.loc[test_index,'y']
    
        X_test = test[FEATURES].reset_index(drop=True).copy()
        # encoder----------------------------------
        
            # INNER K FOLD (TO PREVENT LEAKAGE WHEN USING PRICE)
        kf2 = KFold(n_splits=FOLDS, shuffle=True, random_state=42)   
        for j, (train_index2, test_index2) in enumerate(kf2.split(X_train)):
            print(f" ## INNER Fold {j+1} (outer fold {i+1}) ##")
    
            X_train2 = X_train.loc[train_index2,FEATURES+['y']].copy()
            X_valid2 = X_train.loc[test_index2,FEATURES].copy()
    
            ### FEATURE SET 1 (uses price) ###
            col = "balance"
            tmp = X_train2.groupby(col).y.agg(STATS)
            tmp.columns = [f"TE1_wc_{s}" for s in STATS]
            X_valid2 = X_valid2.merge(tmp, on=col, how="left")
            for c in tmp.columns:
                X_train.loc[test_index2,c] = X_valid2[c].values
    
        ### FEATURE SET 1 (uses price) ###
        col = "balance"
        tmp = X_train.groupby(col).y.agg(STATS)
        tmp.columns = [f"TE1_wc_{s}" for s in STATS]
        X_valid = X_valid.merge(tmp, on=col, how="left")
        X_test = X_test.merge(tmp, on=col, how="left")
    
        ### FEATURE SET 3 (does not use price) ###
        for col in CATS:
            col2 = "balance"
            tmp = X_train.groupby(col)[col2].agg(STATS2)
            tmp.columns = [f"FE3_{col}_wc_{s}" for s in STATS2]
            X_train = X_train.merge(tmp, on=col, how="left")
            X_valid = X_valid.merge(tmp, on=col, how="left")
            X_test = X_test.merge(tmp, on=col, how="left")
        # CC = CATS1+CATS2
        # print(f"Target encoding {len(CC)} features... ",end="")
        # for f,c in enumerate(CC):
        #     if f%10==0: print(f"{f}, ",end="")
        #     # TE0 = TargetEncoder(n_folds=10, smooth=0, split_method='random', stat='mean')
        #     TE0 = ce.TargetEncoder(cols=[c])
        #     X_train[f'{c}_TEE'] = TE0.fit_transform(X_train[c],X_train['y']).astype('float32')
        #     X_valid[f'{c}_TEE'] = TE0.transform(X_valid[c]).astype('float32')
        #     X_test[f'{c}_TEE'] = TE0.transform(X_test[c]).astype('float32')
        # CONVERT TO CATS SO XGBOOST RECOGNIZES THEM
        if mode =='NN':
            for col in X_train.drop('y',axis=1).columns.tolist():
                X_train[col]=X_train[col].fillna(X_train[col].mean())
                X_valid[col]=X_valid[col].fillna(X_valid[col].mean())
                X_test[col]=X_test[col].fillna(X_test[col].mean())
        else:
            X_train[CATS] = X_train[CATS].astype("category")
            X_valid[CATS] = X_valid[CATS].astype("category")
            X_test[CATS] = X_test[CATS].astype("category") 
        # DROP PRICE THAT WAS USED FOR TARGET ENCODING
        X_train = X_train.drop(['y'],axis=1)
       
        # Build model
        if mode ==  'xgb': 
            X_train[CATS] = X_train[CATS].astype("category")
            X_valid[CATS] = X_valid[CATS].astype("category")
            X_test[CATS] = X_test[CATS].astype("category")

            for c in X_train.columns.tolist():
                if c in X_train.select_dtypes(exclude=np.number).columns.tolist():
                    X_train[c]=X_train[c].fillna(X_train[c].mode()[0])
                    X_valid[c]=X_valid[c].fillna(X_valid[c].mode()[0])
                    X_test[c]=X_test[c].fillna(X_test[c].mode()[0])
                else : 
                    X_train[c]=X_train[c].fillna(X_train[c].mean())
                    X_valid[c]=X_valid[c].fillna(X_valid[c].mean())
                X_test[c]=X_test[c].fillna(X_test[c].mean())
            xgb_params = {
                            'objective': 'binary:logistic',
                            'eval_metric': 'auc',
                            'booster': 'gbtree',
                            'max_depth': 6,
                            'learning_rate': 0.05,
                            'subsample': 0.8,
                            'colsample_bytree': 0.8,
                            'min_child_weight': 1,
                            'reg_alpha': 0.1,
                            'reg_lambda': 1,
                            'random_state': 42,
                            "enable_categorical":True,
                            'n_estimators': 10000,
                            'early_stopping_rounds': 100,
                            'verbosity': 1
                        }
            model = XGBClassifier(**xgb_params)
        elif mode ==  'cat':
            model = CatBoostClassifier(
                                allow_writing_files=False,
                                verbose=False,
                                task_type='CPU',
                                loss_function='CrossEntropy',
                                use_best_model=True,
                                cat_features=X_train.columns.to_list(),
                                n_estimators=10000,
                                learning_rate=0.1,
                            )

        elif mode ==  'lgbm':
            model = lgb.LGBMClassifier(
                    objective="binary",
                    metric="auc",
                    n_estimators=20000,
                    learning_rate=0.04967814507666819,
                    num_leaves=120,
                    max_depth=14,
                    min_child_samples=8,
                    subsample=0.9293841645632404,
                    colsample_bytree=0.3000470250632234,
                    reg_alpha=1.2372125517096761,
                    reg_lambda=1.2809008975813085,
                    max_bin=4500,
                    random_state=42,
                    verbosity=-1
                )
        elif mode ==  'rfc':
            model = HistGradientBoostingClassifier(
                learning_rate=0.02,
                max_iter=10_000,
                max_depth=6,
                max_leaf_nodes=31,
                min_samples_leaf=10,
                l2_regularization=1.0,
                early_stopping=True,
                scoring='roc_auc',  # used only when early_stopping=True
                validation_fraction=0.1,
                n_iter_no_change=100,
                categorical_features=CATS,  # only needed if data is NumPy array
                random_state=42,
                verbose=0
            )      
        # TRAIN MODEL
        COLS = X_train.columns
        if mode =='lgbm':
            model.fit(
                     X_train[COLS], y_train,
                    eval_set=[(X_valid[COLS], y_valid)],
                    eval_metric='auc',
                    callbacks=[
                        lgb.early_stopping(500),
                        lgb.log_evaluation(period=500)
                    ]
                )
        elif mode =='rfc':
 
       
            model.fit(
                X_train[COLS], y_train,
            )
      
        elif mode=="cat":
            X_train =X_train.astype('str').fillna(X_train.mode().iloc[0])
            X_valid =X_valid.astype('str').fillna(X_train.mode().iloc[0])
            X_train[COLS]=X_train[COLS]
            model.fit( X_train[COLS], y_train, eval_set=[(X_valid[COLS], y_valid)], early_stopping_rounds=200)
        else:
            model.fit(
                X_train[COLS], y_train,
                eval_set=[(X_valid[COLS], y_valid)],  
                verbose=300,
            )
        
        oof[test_index] = model.predict_proba(X_valid[COLS])[:,1]
        pred += model.predict_proba(X_test[COLS])[:,1]
        
    pred /= FOLDS
    oof_pred_probs[mode]=oof
    test_pred_probs[mode]=pred


    lgb_importances = pd.DataFrame({
    'Feature': X_train.columns.tolist(),
    'Importance': model.feature_importances_
    }).sort_values(by='Importance', ascending=False)
    
    # Set the number of features to display
    n_top_features = 49
    
    plt.figure(figsize=(18, 8))
    sns.set_theme(style="whitegrid")
    
    sns.barplot(
        data=lgb_importances.head(50),
        x='Importance', 
        y='Feature', 
        palette="autumn"
    )
    
    plt.title('LightGBM Top Feature Importances')
    plt.tight_layout()
    plt.show()



    return oof_pred_probs ,test_pred_probs ,lgb_importances

CPU times: user 36 µs, sys: 11 µs, total: 47 µs
Wall time: 52.5 µs


In [40]:
# COMPUTE OVERALL CV SCORE
# true = train.y.values
# for m in [ "lgbm" ]:
    
#     oof_pred_probs ,test_pred_probs ,lgb_importances=train_model(mode=m)
#     print(f"{m} , done")
#     s=roc_auc_score(true,oof_pred_probs[m])
#     print(f"=> {m} Overall CV Score = {s}")
 

In [41]:
# def train_meta_learner(X_meta_train):
#     X_meta_train['mean_pred'] = X_meta_train.mean(axis=1).astype(np.float64)
#     X_meta_train['std_pred'] = X_meta_train.std(axis=1).astype(np.float64)
#     X_meta_train['max_pred'] = X_meta_train.max(axis=1).astype(np.float64)
#     X_meta_train['min_pred'] = X_meta_train.min(axis=1).astype(np.float64)
#     X_meta_train['count_pred'] = X_meta_train.count(axis=1).astype(np.float64)
#     X_meta_train['median_pred'] = X_meta_train.median(axis=1).astype(np.float64)
#     X_meta_train['skew_pred'] = X_meta_train.skew(axis=1).astype(np.float64)
#     X_meta_train['prod_pred'] = X_meta_train.prod(axis=1).astype(np.float64)
    
#     return X_meta_train
 

In [42]:
# oof_pred_probs['xgb']=oof_preds
# test_pred_probs['xgb']=test_preds
# oof_pred_probs=pd.DataFrame(oof_pred_probs)
# test_pred_probs=pd.DataFrame(test_pred_probs)

In [43]:
# oof_pred_probs=train_meta_learner(oof_pred_probs)
# test_pred_probs=train_meta_learner(test_pred_probs)

In [44]:
# X = logit(oof_pred_probs.clip(1e-15, 1-1e-15))
# X_test = logit(test_pred_probs.clip(1e-15, 1-1e-15))


In [45]:
# from sklearn.metrics import roc_auc_score
# from koolbox import Trainer

# class CFG:

    
#     n_folds = 7
#     seed = 42
    
#     cv = StratifiedKFold(n_splits=n_folds, random_state=seed, shuffle=True)
#     metric = roc_auc_score
    
#     n_optuna_trials = 200

In [46]:
# def objective(trial):
#     solver_penalty_options = [
#         ('liblinear', 'l1'),
#         ('liblinear', 'l2'),
#         ('lbfgs', 'l2'),
#         ('lbfgs', None),
#         ('newton-cg', 'l2'),
#         ('newton-cg', None),
#         ('newton-cholesky', 'l2'),
#         ('newton-cholesky', None)
#     ]
#     solver, penalty = trial.suggest_categorical('solver_penalty', solver_penalty_options)
    
#     params = {
#         'random_state': CFG.seed,
#         'max_iter': 1000,
#         'C': trial.suggest_float('C', 0, 1),
#         'tol': trial.suggest_float('tol', 1e-6, 1e-2),
#         'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
#         'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
#         'solver': solver,
#         'penalty': penalty
#     }
    
    
#     trainer = Trainer(
#         LogisticRegression(**params),
#         cv=CFG.cv,
#         metric=CFG.metric,
#         metric_precision=6,
#         use_early_stopping=False,
#         verbose=False,
#         task="binary",
#     )
#     trainer.fit(pd.DataFrame(X), true)
    
#     return np.mean(trainer.fold_scores)

# sampler = optuna.samplers.TPESampler(seed=CFG.seed, multivariate=True, n_startup_trials=CFG.n_optuna_trials // 10)
# study = optuna.create_study(direction='maximize', sampler=sampler)
# study.optimize(objective, n_trials=CFG.n_optuna_trials, n_jobs=-1)
# best_params = study.best_params


In [47]:
# solver, penalty = best_params['solver_penalty']
# lr_params = {
#     'random_state': CFG.seed,
#     'max_iter': 1000,
#     'C': best_params['C'],
#     'tol': best_params['tol'],
#     'fit_intercept': best_params['fit_intercept'],
#     'class_weight': best_params['class_weight'],
#     'solver': solver,
#     'penalty': penalty
# }

In [48]:
# lr_trainer = Trainer(
#     LogisticRegression(**lr_params),
#     cv=CFG.cv,
#     metric=CFG.metric,
#     metric_precision=6,
#     use_early_stopping=False,
#     task="binary",
# )

# lr_trainer.fit(pd.DataFrame(X), true)

# scores["LogisticRegression"] = lr_trainer.fold_scores
# lr_test_pred_probs = lr_trainer.predict(X_test)
# print(np.mean(scores["LogisticRegression"]))

In [49]:
# sub = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")
# sub['y']=lr_test_pred_probs

# sub.to_csv("submission.csv",index=False)
# sub.head()

In [50]:
# lr_ens=0.8*lr_test_pred_probs+0.2*prd_ens
# sub = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")
# sub['y']=lr_test_pred_probs

# sub.to_csv("submission.csv",index=False)
# # sub.head()
# sub2=pd.DataFrame(oof_pred_probs)
# sub2.to_csv("oof_pred_probs.csv",index=False)
# lgb_importances.to_csv("lgb_importances.csv",index=False)

